In [3]:
# import packages 
import sys
dataFolder = '../../../../data/fields/'
sys.path.append(dataFolder)
from Auxiliary.helper import *
%matplotlib inline
from rocksdbutils_copy import *

def getFilelist(originpath, ftyp):
    files = os.listdir(originpath)
    out   = []
    for i in files:
        if i.split('.')[-1] in ftyp:
            if originpath.endswith('/'):
                out.append(originpath + i)
            else:
                out.append(originpath + '/' + i)
        # else:
        #     print("non-matching file - {} - found".format(i.split('.')[-1]))
    return out


In [5]:
# Set up export directory
export_dir = dataFolder + "output/nc_to_tif"
origins = getFilelist(dataFolder + "ai4boundaries/sentinel2/masks/LU" , 'tif')
origins.sort()

for modi in ['train', 'valid']:
    db_path = dataFolder + 'output/rocks_db/FORCE.db/' + modi + '.db'
        
    # Initialize RocksDBReader
    reader = RocksDBReader(flname_db=db_path)
  
    # Iterate over data
    for idx in range(len(reader.keys)):
        # Get inputs and labels (assuming the database contains both)
        data = reader.get_inputs_labels(idx)  # This returns a list of arrays for inputs and labels

        # Process each column family
        for i, (cf_name, array) in enumerate(zip(reader.cf_names, data)):

            cf_name_str = cf_name.decode('utf-8')  # Convert bytes to string for readable file names
            print(f"cf name is: {cf_name_str} and shape is: {array.shape}")
        
            if cf_name_str == 'inputs':
                # Export to TIFF
                for t_idx in range(array.shape[0]):
                    out_file = f"{export_dir}/{cf_name_str}_image_{idx}_{modi}_{t_idx}.tif"
                    
                    with rasterio.open(
                        out_file,
                        "w",
                        driver="GTiff",
                        height=array.shape[2],
                        width=array.shape[3],
                        count=array.shape[1],
                        dtype=array.dtype,
                        crs=rasterio.open(origins[0]).crs,
                        transform=rasterio.open(origins[0]).transform,
                        ) as dst:
                                for r_idx in range(array.shape[1]):
                                    dst.write(array[t_idx, r_idx, :, :], r_idx + 1)

                    print(f"Exported {cf_name_str} index {idx} to {out_file}")
    
            else:
                 # Export to TIFF
                out_file = f"{export_dir}/{cf_name_str}_image_{idx}_{modi}.tif"
                
                with rasterio.open(
                    out_file,
                    "w",
                    driver="GTiff",
                    height=array.shape[1],
                    width=array.shape[2],
                    count=array.shape[0],
                    dtype=array.dtype,
                    crs=rasterio.open(origins[0]).crs,
                    transform=rasterio.open(origins[0]).transform,
                    ) as dst:
                            for r_idx in range(array.shape[0]):
                                dst.write(array[r_idx, :, :], r_idx + 1)

                print(f"Exported {cf_name_str} index {idx} to {out_file}")
                print(array.shape)
        if idx == 50:
            break


b'0'
b'inputs'


ValueError: cannot reshape array of size 49152 into shape (4,6,128,128)